### Runnables:

#### RunnableSequence

- Serve para invocar uma cadeia sequencia onde a saída de um componente serve de entrada para o próximo componente.
- **RunnableSequence** é o operador de composição mais importante no LangChain, pois é usado em praticamente todas as cadeias.
-  Um RunnableSequence pode ser instanciado diretamente ou, mais comumente, usando o operador | , onde os operandos esquerdo ou direito (ou ambos) devem ser um componente com base Runnable.

In [1]:
%run ../helpers/00-llm.ipynb

In [2]:

# Importa conectando com o cliente OpenAI
from helpers.llm import initialize_llm, logger, pretty_print

llm, _, _ = initialize_llm()

INFO:helpers.llm:Using AzureOpenAI.


#### RunnableLambda

- RunnableLambda converte um python `callable` (pode-se entender aqui, a principio, como qualquer 'função' python) em um Runnable.
- Esse executável envolve o `callable` para que você possa utilizar ele em uma sequencia de componentes de LangChain

Exemplo:

Imagine que eu quero usar na minha Chain a seguinte função personalizada: `add_one`. Ela no caso não seque os padrões do protocolo `runnable`, ou seja, precisamos transformar ela em um executável langchain. Dessa forma encapsulamos nossa função em um `RunnableLambda`. Em teoria estamos transformando nossa função criada em uma função do estilo `lambda` do python.


In [5]:
from langchain_core.runnables import RunnableLambda

# Função customizada
def adicionar(x: int) -> int:
    return x + 1

runnable = RunnableLambda(adicionar) # agora eu implementei os protocolos padrões do langchain, exemplo o invoke.

resposta = runnable.invoke(1) # returns 2

print(resposta)

2


#### RunnableSequence

In [9]:
# Veja o que é o RunnableLambda abaixo.
from langchain_core.runnables import RunnableLambda,RunnableSequence

def somar(x: int) -> int:
    return x + 1

def multiplicar(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(somar)
runnable_2 = RunnableLambda(multiplicar)


sequence = runnable_1 | runnable_2
# Or equivalently:
#sequence = RunnableSequence(first=runnable_1, last=runnable_2)
resposta = sequence.invoke(1)
print(resposta)

4


#### RunnableParallel

- Serve para executar componentes (ou operações) de forma paralela, quando ambos recebem o mesmo tipo de entrada. 
- Normalmente é representado por um dicionário usando chaves "{ ... }", desde que não seja inicio de Chain, neste caso precisamos declarar usando `RunnableParallel` literalmente ao invés de usar dicionários .

In [ ]:
from langchain_core.runnables import RunnableLambda,RunnableParallel

def dividir(x: int) -> int:
    return x / 2

runnable_1 = RunnableLambda(somar)
runnable_2 = RunnableLambda(multiplicar)
runnable_3 = RunnableLambda(dividir)

# sequence = runnable_1 | {  # o dicionário aqui é entendido como 'RunnableParallel'
#     "multiplicar": runnable_2,
#     "dividir": runnable_3,
# }
# Ou usando o equivalente:
# sequence = runnable_1 | RunnableParallel(
#     {"multiplicar": runnable_2, "dividir": runnable_3}
# )
# Ou também o equivalente:
sequence = runnable_1 | RunnableParallel(
    multiplicar=runnable_2,
    dividir=runnable_3,
)

resposta = sequence.invoke(2)
print(resposta)

{'multiplicar': 6, 'dividir': 1.5}


#### RunnablePassthrough

- Executável para passar entradas inalteradas ou com chaves adicionais.
- Ao compor cadeias com várias etapas, às vezes você vai querer passar dados de etapas anteriores inalterados para uso como entrada para uma etapa posterior. A classe  `RunnablePassthrough` permite que você faça exatamente isso, e é tipicamente usada em conjunto com um `RunnableParallel` para passar dados para uma etapa posterior em suas cadeias construídas.

Exemplo: Sem transformar a entrada. 


In [11]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough ()

# Independente de quantas vezes você "passar o resultado para frente", a entrada não é alterada.

resposta = chain.invoke("Olá")

print(resposta) # retorna Olá

Olá


Exemplo: Transformando a entrada.


In [12]:
from langchain_core.runnables import RunnablePassthrough

def entrada_para_letras_maiusculas(entrada: str):
	saida = entrada.upper()
	return saida

chain = RunnablePassthrough() | RunnableLambda(entrada_para_letras_maiusculas) | RunnablePassthrough ()

# Neste caso vamos receber a entrada do usuário, passar para a função 'entrada_para_letras_maiusculas', transformar olá -> OLÁ e passar para frente.

resposta = chain.invoke("olá")

print(resposta) # retorna OLÁ

OLÁ


#### Operador Assign
- Normalmente é usado com a combinação `RunnablePassthrough.assign()` para que você possa criar uma saída em dicionário que será usada como entrada ao próximo componente com tal que você tenha a entrada inalterada e uma outra chave com a entrada transformada.

Exemplo:

In [13]:
# https://python.langchain.com/docs/how_to/assign/

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    modified=lambda x: x["num"] + 1,
)

runnable = RunnablePassthrough() | RunnablePassthrough.assign(multiplica_3=lambda x: x["num"] * 3)

resposta = runnable.invoke({"num": 1})

print(resposta) # Saida {"num": 1, "multiplica_3": 3}


{'num': 1, 'multiplica_3': 3}


### Desafio com Runnables:

Processo:

1) Receber a entrada do tipo {"input": "SRP — Princípio da Responsabilidade Única."}  e passar para frente (testar uso do RunnablePassthrough)

2) Quando eu receber a entrada de (1) eu gostaria de criar um dicionário mantendo a entrada de (1) intacta, mas criando 
uma chave nova ("num_caract") tal que receba a entrada de (1) e conte o total de caracteres.

3) Usando a saída de (2) quero paralelizar a entrada em dois processos:
    1) primeiro, pegando a entrada textual e adicionando a frase " É um dos grandes principios do SOLID!" na chave chamada "transformar_entrada"
    2) segundo não farei nada, apenas passarei para frente a entrada sem qualquer alteração na chave "passa_para_frente".

4) Por fim, vou passar para frente a combinação do processo paralelo e imprimir o resultado.